Exploratory Data Analysis

In [1]:
import pandas as pd

cashrequest_df = pd.read_csv("project_dataset/cashreq-clean.csv")
fees_and_cohorts_df = pd.read_csv("project_dataset/fees-with-cohort-and-user-id-clean.csv")

C:\Users\edumc\AppData\Local\Temp\ipykernel_17708\1667466518.py:4: DtypeWarning: Columns (29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  fees_and_cohorts_df = pd.read_csv("project_dataset/fees-with-cohort-and-user-id-clean.csv")


In [ ]:
# Edu: EXTRA metric 4:
# relacion volumen de cash reques contra reco creation y reco resolve. Los que piden poco han tenido incidencias más largas (recovery)


# "We analyzed the relationship between the volume of cash requests and the creation and resolution of recovery cases.
# Hypothesis to test: Users with fewer requests tend to experience longer recovery times.
#New Metric: XXXX
# Columns: 



In [ ]:
#Function for showing data with seaborn:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.countplot(data=incident_feed, x='cohort', palette='viridis')  # <- Notice y='Country' instead of x
incident_feed.pivot.plot(kind='bar', stacked=True, colormap='viridis')
plt.title('Fee Incidents by Cohort')
plt.xlabel('Cohort')
plt.ylabel('Fee Incidents')
plt.xticks(rotation=45, ha='right')  # Rotate x labels for better visibility
plt.tight_layout()
plt.show()   
    
    
    

SyntaxError: invalid syntax (300597740.py, line 1)